Single Race Exploratory Data Analysis -

The purpose of this notebook is to explore and wrangle data from the Fast-F1 API for a single race. This serves as a starting point to establish a clean workflow for data acquisition, preparation, and early exploration before scaling to a multi-race analysis. I will focus on building a data dictionary, assessing data quality, generating descriptive statistics, and creating preliminary visualizations to understand the structure and sufficiency of the data. The outcome of this notebook will be a reproducible workflow for future feature engineering and multi-race EDA.

The code below adds the parent directory to Python’s module search path and configures logging to suppress all FastF1 logs below the warning level. This will enable subsequent code blocks that use imports to work seamlessly and keep my resulting code compilations clean and easy to read.

In [39]:
import sys
import os
import logging

# Add the root directory to sys.path
root = os.path.abspath("..")
sys.path.append(root)

# Suppress FastF1 info logs globally
logging.getLogger('fastf1').setLevel(logging.WARNING)

In this section, I import Python libraries for data visualization, numerical analysis, and working with Pandas dataframes that the FastF1 API is primarily structured with. I also import custom modules for accessing preprocessed F1 data and constants. To support full visibility into the datasets without truncation, I configure Pandas display options to show all rows and columns.

In [52]:
from src.data import f1data
from src.utils import f1constants

from matplotlib.collections import LineCollection
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Set pandas display options
pd.set_option('display.max_rows', None)  # show all rows
pd.set_option('display.max_columns', None)  # show all columns


The following code initializes a single F1 race session by defining parameters such as year, location, and session type. These values are passed into the custom F1Session class (from f1data.py), which creates a session object built on top of FastF1. This object provides access to key race data as well as custom functions I’ve implemented.

The session parameters were chosen to best match Tier 1 control qualities:

- Weather: Abu Dhabi (dry conditions)
- Max Speed: C5 Ultra Soft Tires
- Minimize Outliers: Top 3 valid laps from Q3
- Traffic: Remove tow laps to avoid slipstream bias

In [53]:
# Define session parameters
year = 2024
grand_prix = f1constants.F1Constants.LOCATIONS["Abu Dhabi"]
session_type = f1constants.F1Constants.SESSIONS["Q"]

# Call session object
session = f1data.F1Session(year, grand_prix, session_type)

All drivers who participated in the Q2 and Q3 sessions will be analyzed. Drivers will be assigned variables and identified by their three-letter codes for further analysis. Q3 data will be prioritized over Q2 if available for that driver.

In [54]:
# Drivers from each team that participated in Q2 and Q3 accessed by their three-letter code

norris_mcLaren = 'NOR'
piastri_mcLaren = 'PIA'

verstappen_redBull = 'VER'
perez_redBull = 'PER'

sainz_ferrari = 'SAI' 
leclerc_ferrari = 'LEC'

gasly_alpine = 'GAS' 
ocon_alpine = 'OCO'

bottas_alfaRomeo = 'BOT' 
tsunoda_alfaRomeo = 'TSU'

alonso_astonMartin = 'ALO' 
stroll_astonMartin = 'STR'

russell_mercedes = 'RUS'

hulkenberg_haas = 'HUL'


In [ ]:
# Q3 telemetry data for Lando Norris, McLaren
q1, q2, q3 = session.get_laps(norris_mcLaren).split_qualifying_sessions()

norris_q3_telemetry = session.get_telemetry(q3)

norris_q3_telemetry

In [ ]:
# Q3 telemetry data for Oscar Piastri, McLaren
q1, q2, q3 = session.get_laps(piastri_mcLaren).split_qualifying_sessions()

piastri_q3_telemetry = session.get_telemetry(q3)

piastri_q3_telemetry

In [ ]:
# Q3 telemetry data for Max Verstappen, Red Bull
q1, q2, q3 = session.get_laps(verstappen_redBull).split_qualifying_sessions()

verstappen_q3_telemetry = session.get_telemetry(q3)

verstappen_q3_telemetry

In [ ]:
# Q3 telemetry data for Sergio Perez, Red Bull
q1, q2, q3 = session.get_laps(perez_redBull).split_qualifying_sessions()

perez_q3_telemetry = session.get_telemetry(q3)

perez_q3_telemetry

In [ ]:
# Q3 telemetry data for Carlos Sainz, Ferrari
q1, q2, q3 = session.get_laps(sainz_ferrari).split_qualifying_sessions()

sainz_q3_telemetry = session.get_telemetry(q3)

sainz_q3_telemetry

In [ ]:
# Q2 telemetry data for Charles Leclerc, Ferrari
q1, q2, q3 = session.get_laps(leclerc_ferrari).split_qualifying_sessions()

leclerc_q2_telemetry = session.get_telemetry(q2)

leclerc_q2_telemetry

In [ ]:
# Q3 telemetry data for Pierre Gasly, Alpine
q1, q2, q3 = session.get_laps(gasly_alpine).split_qualifying_sessions()

gasly_q3_telemetry = session.get_telemetry(q3)

gasly_q3_telemetry

In [ ]:
# Q2 telemetry data for Esteban Ocon, Alpine
q1, q2, q3 = session.get_laps(ocon_alpine).split_qualifying_sessions()

ocon_q2_telemetry = session.get_telemetry(q2)

ocon_q2_telemetry

In [ ]:
# Q3 telemetry data for Valtteri Bottas, Alfa Romeo
q1, q2, q3 = session.get_laps(bottas_alfaRomeo).split_qualifying_sessions()

bottas_q3_telemetry = session.get_telemetry(q3)

bottas_q3_telemetry

In [ ]:
# Q2 telemetry data for Yuki Tsunoda, Alfa Romeo
q1, q2, q3 = session.get_laps(tsunoda_alfaRomeo).split_qualifying_sessions()

tsunoda_q2_telemetry = session.get_telemetry(q2)

tsunoda_q2_telemetry

In [ ]:
# Q3 telemetry data for Fernando Alonso, Aston Martin
q1, q2, q3 = session.get_laps(alonso_astonMartin).split_qualifying_sessions()

alonso_q3_telemetry = session.get_telemetry(q3)

alonso_q3_telemetry

In [ ]:
# Q2 telemetry data for Lance Stroll, Aston Martin
q1, q2, q3 = session.get_laps(stroll_astonMartin).split_qualifying_sessions()

stroll_q2_telemetry = session.get_telemetry(q2)

stroll_q2_telemetry

In [ ]:
# Q3 telemetry data for George Russell, Mercedes
q1, q2, q3 = session.get_laps(russell_mercedes).split_qualifying_sessions()

russell_q3_telemetry = session.get_telemetry(q3)

russell_q3_telemetry

In [ ]:
# Q3 telemetry data for Nico Hulkenberg, Haas
q1, q2, q3 = session.get_laps(hulkenberg_haas).split_qualifying_sessions()

hulkenberg_q3_telemetry = session.get_telemetry(q3)

hulkenberg_q3_telemetry